In [ ]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.1-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia"'

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed Preferences ───── v1.2.2
  Installed IJulia ────────── v1.23.2
  Installed VersionParsing ── v1.2.0
  Installed libsodium_jll ─── v1.0.19+0
  Installed JLLWrappers ───── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed Conda ─────────── v1.5.2
  Installed Parsers ───────── v1.1.0
  Installed ZMQ ───────────── v1.2.1
  Installed SoftGlobalScope ─ v1.1.0
  Installed MbedTLS ───────── v1.0.3
  Installed JSON ──────────── v0.21.1
  Installed Artifacts ─────── v1.3.0
  Installed ZeroMQ_jll ────── v4.3.2+6
  Installed TOML ──────────── v1.0.3
######################################################################## 100.0%
######################################################################## 100.0%
######################################################

上記をshift+enterで実行後、「ランタイム」->「ランタイムのタイプを変更」->「保存」を押してください。

In [ ]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, broadwell)


# Flux.jl

In [ ]:
# パッケージの追加
import Pkg; Pkg.add("Flux")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
  Installed ZygoteRules ────────────────── v0.2.1
  Installed CodecZlib ──────────────────── v0.7.0
  Installed CompilerSupportLibraries_jll ─ v0.3.4+0
  Installed StatsAPI ───────────────────── v1.0.0
  Installed TimerOutputs ───────────────── v0.5.8
  Installed NaNMath ────────────────────── v0.3.5
  Installed ExprTools ──────────────────── v0.1.3
  Installed Flux ───────────────────────── v0.12.1
  Installed DataAPI ────────────────────── v1.6.0
  Installed AbstractTrees ──────────────── v0.3.4
  Installed Media ──────────────────────── v0.5.0
  Installed IRTools ────────────────────── v0.4.2
  Installed Zlib_jll ───────────────────── v1.2.11+18
  Installed CommonSubexpressions ───────── v0.3.0
  Installed Colors ─────────────────────── v0.12.8
  Installed Juno ───────────────────────── v0.8.4
  Installed Scratch ────────────────────── v1.0.3
  Installed MacroTools ─────────────────── v0.5.6
  Inst

In [ ]:
# パッケージの呼び出し
using Flux

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1278


######################################################################### 100.0%


########################################################################   99.5%


curl: (92) HTTP/2 stream 1 was not closed cleanly: INTERNAL_ERROR (err 2)


######################################################################### 100.0%


######################################################################### 100.0%


## DataLoader

In [ ]:
Xtrain = rand(10, 100)
Ytrain = rand(100)
train_loader = Flux.Data.DataLoader((Xtrain, Ytrain), batchsize=2, shuffle=true)
for epoch in 1:100
    for (x, y) in train_loader
        @assert size(x) == (10, 2)
        @assert size(y) == (2,)
    end
end

## レイヤー・モデル定義

In [ ]:
# 線形レイヤーを定義
function linear(in, out)
  W = randn(out, in)
  b = randn(out)
  x -> W * x .+ b
end

linear1 = linear(5, 3)
linear2 = linear(3, 2)

# 間にシグモイド（σ）のような非線形性を持つ2つの線形レイヤー
model(x) = linear2(σ.(linear1(x)))

model(rand(5)) 

2-element Array{Float64,1}:
 -0.16259799675673933
 -0.10064123152334448

In [ ]:
# Denseレイヤーを定義
layer3 = Flux.Dense(10, 5, σ)

Dense(10, 5, σ)

In [ ]:
# レイヤーを重ねてモデルを作る
model2 = Flux.Chain(
  Flux.Dense(10, 5, σ),
  Flux.Dense(5, 2),
  Flux.softmax)

model2(rand(10))

2-element Array{Float64,1}:
 0.36457995972186624
 0.6354200402781338

## 損失関数

In [ ]:
y_model = [1.1, 1.9, 3.1];
y_true = 1:3;
Flux.mse(y_model, y_true)

0.010000000000000018

In [ ]:
function loss(ŷ, y)
  Flux.mean((y .- ŷ).^2)
end

loss(y_model, y_true)

0.010000000000000018

## オプティマイザー

In [ ]:
Xtrain = rand(10, 100)
Ytrain = rand(100)
train_loader = Flux.Data.DataLoader((Xtrain, Ytrain), batchsize=2, shuffle=true)

model3 = Flux.Chain(
  Flux.Dense(10, 5, σ),
  Flux.Dense(5, 2),
  Flux.softmax)

loss(x, y) = Flux.mse(model3(x), y)

opt = ADAM()
@time Flux.train!(loss, params(model3), train_loader, opt)

  0.065797 seconds (110.76 k allocations: 5.563 MiB)
